In [1]:
import os
import json
import pprint
import docx

from dotenv import load_dotenv

from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import Docx2txtLoader
from langchain_community.vectorstores import FAISS
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

In [2]:
load_dotenv()
open_api_key = os.getenv("OPENAI_API_KEY")
if not open_api_key:
    raise ValueError("OPENAI_API_KEY is not set")

In [3]:
loader = TextLoader("data/data_arshin_mal_alan.txt", encoding="utf-8")
docs = loader.load()

In [4]:
loader = Docx2txtLoader("data/data_arshin_mal_alan.docx")
docs = loader.load()

In [5]:
generative_llm = ChatOpenAI(model="gpt-4o", temperature=0)

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200, 
    chunk_overlap=50,
    # separators=["\n"]
)   

splits = text_splitter.split_documents(docs)
splits

[Document(metadata={'source': 'data/data_arshin_mal_alan.docx'}, page_content='Arşın mal alan'),
 Document(metadata={'source': 'data/data_arshin_mal_alan.docx'}, page_content='"Arşın Mal Alan" filmi Üzeyir Hacıbəyovun eyniadlı musiqili komediyasına əsaslanır. Bu əsər Azərbaycan kinosunda bir neçə dəfə ekranlaşdırılmışdır. Ən məşhur versiyalardan ikisi 1945-ci ildə Rza'),
 Document(metadata={'source': 'data/data_arshin_mal_alan.docx'}, page_content='Ən məşhur versiyalardan ikisi 1945-ci ildə Rza Təhmasib və Nikolay Leşşenko, digəri isə 1965-ci ildə Tofiq Tağızadə tərəfindən çəkilmişdir.'),
 Document(metadata={'source': 'data/data_arshin_mal_alan.docx'}, page_content='Filmin Tarixi və Versiyaları'),
 Document(metadata={'source': 'data/data_arshin_mal_alan.docx'}, page_content='"Arşın Mal Alan" operettası ilk dəfə 1913-cü ildə səhnəyə qoyulmuşdur. Bu əsər Azərbaycan teatrının klassik nümunələrindən hesab olunur. Filmin əsas mövzusu XX əsrin əvvəlində Bakıda cərəyan edir.'),
 Document(meta

In [7]:
vectorstore = FAISS.from_documents(
    documents=splits, 
    embedding=OpenAIEmbeddings(model="text-embedding-3-large")
)
retriever = vectorstore.as_retriever()
# retriever=vectorstore.as_retriever(search_type="similarity_score_threshold", search_kwargs={
#                               'score_threshold': 0.5})

model = HuggingFaceCrossEncoder(model_name="cross-encoder/ms-marco-MiniLM-L-2-v2")
compressor = CrossEncoderReranker(model=model, top_n=3)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

c:\Users\murad\AppData\Local\Programs\Python\Python312\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

c:\Users\murad\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\murad\.cache\huggingface\hub\models--cross-encoder--ms-marco-MiniLM-L-2-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/62.5M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [8]:
# Some useful db operations

# vectorstore.index.reset()
# vectorstore.index.ntotal
# vectorstore.index.reconstruct(0)

In [9]:
compression_retriever

ContextualCompressionRetriever(base_compressor=CrossEncoderReranker(model=HuggingFaceCrossEncoder(client=<sentence_transformers.cross_encoder.CrossEncoder.CrossEncoder object at 0x000001A11D0F5670>, model_name='cross-encoder/ms-marco-MiniLM-L-2-v2', model_kwargs={}), top_n=3), base_retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001A17987C3B0>, search_kwargs={}))

In [10]:
### Contextualize question ###
contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [11]:
history_aware_retriever = create_history_aware_retriever(
    generative_llm, compression_retriever, contextualize_q_prompt
)

In [12]:
### Answer question ###
qa_system_prompt = """You are an assistant for question-answering tasks. \
Use ONLY the provided retrieved context to answer the question. \
If the context does not contain relevant information, simply respond with: \
"I don’t know based on the given information." \

Retrieved context: 
{context}"""

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(generative_llm, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [13]:
### Statefully manage chat history ###
store = {}
session_id = "chatbot_first_session"

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [14]:
conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [15]:
store

{}

In [16]:
conversational_rag_chain.invoke(
    {"input": "Arşın mal alan barədə məlumat ver"},
    config={
        "configurable": {"session_id": session_id}
    }, 
)["answer"]

'"Arşın Mal Alan" filmi Azərbaycan mədəniyyətinə və kinematoqrafiyasına əhəmiyyətli töhfə vermişdir. Bu film Azərbaycanın milli kimliyini və mədəni irsini dünya miqyasında tanıtmışdır. Filmin musiqisi də bu prosesdə mühüm rol oynayır. Son illərdə filmin yenidən bərpası məsələsi aktualdır və rəngləndirilməsi, yeni formatda nümayişi planlaşdırılır ki, bu da filmin daha geniş auditoriya tərəfindən qəbuluna kömək edəcək. Həmçinin, filmə həsr olunmuş mədəni tədbirlər keçirilir ki, bu da onun mədəni əhəmiyyətini daha da artırır.'

In [17]:
conversational_rag_chain.invoke(
    {"input": "Son sualımı təkrarla?"},
    config={
        "configurable": {"session_id": session_id}
        },
        
)["answer"] 

'"Arşın mal alan" barədə məlumat ver.'

In [18]:
pprint.pprint(store)

{'chatbot_first_session': InMemoryChatMessageHistory(messages=[HumanMessage(content='Arşın mal alan barədə məlumat ver', additional_kwargs={}, response_metadata={}), AIMessage(content='"Arşın Mal Alan" filmi Azərbaycan mədəniyyətinə və kinematoqrafiyasına əhəmiyyətli töhfə vermişdir. Bu film Azərbaycanın milli kimliyini və mədəni irsini dünya miqyasında tanıtmışdır. Filmin musiqisi də bu prosesdə mühüm rol oynayır. Son illərdə filmin yenidən bərpası məsələsi aktualdır və rəngləndirilməsi, yeni formatda nümayişi planlaşdırılır ki, bu da filmin daha geniş auditoriya tərəfindən qəbuluna kömək edəcək. Həmçinin, filmə həsr olunmuş mədəni tədbirlər keçirilir ki, bu da onun mədəni əhəmiyyətini daha da artırır.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Son sualımı təkrarla?', additional_kwargs={}, response_metadata={}), AIMessage(content='"Arşın mal alan" barədə məlumat ver.', additional_kwargs={}, response_metadata={})])}


In [19]:
for msg in store[session_id].messages:
    print(f"{msg.type.upper()}: {msg.content}")

response = conversational_rag_chain.invoke(
    {"input": "Can you list all questions that I've addressed to you so far?"},
    config={"configurable": {"session_id": session_id}},
)["answer"]


HUMAN: Arşın mal alan barədə məlumat ver
AI: "Arşın Mal Alan" filmi Azərbaycan mədəniyyətinə və kinematoqrafiyasına əhəmiyyətli töhfə vermişdir. Bu film Azərbaycanın milli kimliyini və mədəni irsini dünya miqyasında tanıtmışdır. Filmin musiqisi də bu prosesdə mühüm rol oynayır. Son illərdə filmin yenidən bərpası məsələsi aktualdır və rəngləndirilməsi, yeni formatda nümayişi planlaşdırılır ki, bu da filmin daha geniş auditoriya tərəfindən qəbuluna kömək edəcək. Həmçinin, filmə həsr olunmuş mədəni tədbirlər keçirilir ki, bu da onun mədəni əhəmiyyətini daha da artırır.
HUMAN: Son sualımı təkrarla?
AI: "Arşın mal alan" barədə məlumat ver.


In [20]:
retriever.invoke(
    input="Do you know anything about AILAB", 
    config={}
)

[Document(id='0d7094fd-8a4b-4f09-a0ec-711969cff58f', metadata={'source': 'data/data_arshin_mal_alan.docx'}, page_content='IMDb: ID1573789'),
 Document(id='8e7f580a-6c80-4d8a-af39-2be216399115', metadata={'source': 'data/data_arshin_mal_alan.docx'}, page_content='Mədəni Tədbirlər\n\n"Arşın Mal Alan" filminə həsr olunmuş mədəni tədbirlər keçirilir. Bu tədbirlər filmin mədəni əhəmiyyətini daha da artırır.\n\nTəhsil Prosesində İstifadə'),
 Document(id='0b381dfb-d94b-4793-9d88-5ce57ca901c2', metadata={'source': 'data/data_arshin_mal_alan.docx'}, page_content='Arşın mal alan'),
 Document(id='dea63a8f-3952-426d-9046-eac0168626e4', metadata={'source': 'data/data_arshin_mal_alan.docx'}, page_content='Baş Rollarda: Həsən Məmmədov, Leyla Şıxlinskaya, Ağadadaş Qurbanov, Nəcibə Məlikova, Hacımurad Yegizarov, Xuraman Hacıyeva, Tələt Rəhmanov, Səfurə İbrahimova\n\n\nOperator: İlya Minkovetski')]

In [23]:
query = "musiqini kim bəstələyib"
docs_and_scores = vectorstore.similarity_search_with_score(query, k=5)

for doc, score in docs_and_scores:
    print("----")
    print("Document:\n", doc.page_content)
    print("Metadata:", doc.metadata)
    print("Score:", score)

----
Document:
 Musiqi
Metadata: {'source': 'data/data_arshin_mal_alan.docx'}
Score: 0.7893481
----
Document:
 Məzmun
Metadata: {'source': 'data/data_arshin_mal_alan.docx'}
Score: 0.8806992
----
Document:
 daha geniş auditoriya tərəfindən qəbuluna kömək edəcəkdir.
Metadata: {'source': 'data/data_arshin_mal_alan.docx'}
Score: 1.1139009
----
Document:
 Filmin Təsiri
Metadata: {'source': 'data/data_arshin_mal_alan.docx'}
Score: 1.1199505
----
Document:
 dünya miqyasında tanıtmışdır. Filmin musiqisi və məzmunu izləyicilərə Azərbaycanın zəngin mədəni irsini göstərir.
Metadata: {'source': 'data/data_arshin_mal_alan.docx'}
Score: 1.1260152
